<a href="https://colab.research.google.com/github/khanhthanhh9/Deep-Learning-Model/blob/main/2_Korea_Chinese_VN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 8.7MB/s 
     |████████████████████████████████| 204kB 39.5MB/s 
     |████████████████████████████████| 1.2MB 40.0MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
Mounted at /content/gdrive


In [ ]:
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
key = os.environ.get('AZURE_SEARCH_KEY', '455fc18d0b0c42b384ee21cabcc4dc31')

In [ ]:
results = search_images_bing(key, 'Vietnam Women')
ims = results.attrgot('contentUrl')
len(ims)

150

In [ ]:
doc(L.attrgot)

In [ ]:
girl_types = 'vietnam','chinese','korean'
path = Path('women')

In [ ]:
if not path.exists():
    path.mkdir()
    for o in girl_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o} women')
        download_images(dest, urls=results.attrgot('contentUrl'))

In [ ]:
fns = get_image_files(path)
fns

(#430) [Path('women/vietnam/00000098.jpg'),Path('women/vietnam/00000051.jpg'),Path('women/vietnam/00000026.jpg'),Path('women/vietnam/00000032.jpg'),Path('women/vietnam/00000105.jpg'),Path('women/vietnam/00000072.jpg'),Path('women/vietnam/00000022.jpg'),Path('women/vietnam/00000019.jpg'),Path('women/vietnam/00000020.jpg'),Path('women/vietnam/00000092.jpg')...]

In [ ]:
failed = verify_images(fns)
failed

(#3) [Path('women/vietnam/00000059.jpg'),Path('women/vietnam/00000060.jpg'),Path('women/korean/00000014.jpg')]

In [ ]:
failed.map(Path.unlink);

In [ ]:
girls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
dls = girls.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=4, nrows=5)

In [ ]:
girls = girls.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.7),
    batch_tfms=aug_transforms())
dls = girls.dataloaders(path)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(8)

epoch,train_loss,valid_loss,error_rate,time
0,2.020743,2.015229,0.630952,00:56


epoch,train_loss,valid_loss,error_rate,time
0,1.545161,1.275471,0.511905,01:10
1,1.414719,1.163438,0.511905,01:10
2,1.306768,1.277244,0.511905,01:10
3,1.191957,1.367140,0.535714,01:10
4,1.108944,1.398963,0.511905,01:10
5,1.014657,1.341737,0.500000,01:10
6,0.929621,1.340562,0.464286,01:10
7,0.884190,1.339980,0.464286,01:10


In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

NameError: ignored

In [ ]:
interp.plot_top_losses(20, nrows=4)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner


In [ ]:
for idx in cleaner.delete(): cleaner.fns[idx].unlink()


In [ ]:
learn.export()

In [ ]:
path = Path()
path.ls(file_exts='.pkl')

(#1) [Path('export.pkl')]

In [ ]:
learn_inf = load_learner(path/'export.pkl')

In [ ]:
btn_upload = widgets.FileUpload()
btn_upload


FileUpload(value={}, description='Upload')

In [ ]:
img = PILImage.create(btn_upload.data[-1])
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
pred,pred_idx,probs = learn_inf.predict(img)
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
btn_run = widgets.Button(description='Classify')

def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
    


In [ ]:
btn_run.on_click(on_click_classify)

In [ ]:
btn_upload = widgets.FileUpload()

In [ ]:
VBox([widgets.Label('Select your grill!'), 
      btn_upload, btn_run, out_pl, lbl_pred])